This file needs requires external data!

(Run on workstation)

In [1]:
import xarray as xr
import glob
import numpy as np

import sys

sys.path.append('../src')
import utils as ut

In [2]:
run = 'n011'

In [3]:
fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*.nc'))

year0 = np.array([int(fname[-27:-23]) for fname in fnames])
year1 = np.array([int(fname[-18:-14]) for fname in fnames])
nyears = sum(year1+1-year0)

In [4]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset(fnames[0])
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed
ds.close()
secs = (time_bnds[:,1]-time_bnds[:,0]).values / np.timedelta64(1, 's')

In [5]:
#Get weighted mask for basin averages
lons = np.repeat(lon[np.newaxis,:,:],len(levmid),axis=0)
lats = np.repeat(lat[np.newaxis,:,:],len(levmid),axis=0)
mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],len(ut.basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bas in enumerate(ut.basin):
    mm = np.zeros(lons.shape)
    if bas=='East Ant.':
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
        depp = 369
    if bas=='Ross':
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
        depp = 312        
    if bas=='Amundsen':
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
        depp = 305
    if bas=='Weddell':
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
        depp = 420
    if bas=='Peninsula':
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        depp = 420
        
    z0 = depp-50.
    i0 = np.argmax(lev[:,1]>z0)
    mm[:i0,:,:] = 0
    w0 = (lev[i0,1]-z0)/(lev[i0,1]-lev[i0,0])
    mm[i0,:,:] = w0*mm[i0,:,:]
    for j in range(0,lon.shape[0]):
        for i in range(0,lon.shape[1]):
            if np.nansum(thick[0,i0:,j,i]) == 0:
                continue
            z1 = depp+50.
            i1 = np.argmin(lev[:,1]<z1)
            w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
            mm[i1,j,i] = w1*mm[i1,j,i]
            mm[i1+1:,j,i] = 0
    mask[b,:,:,:] = mm*np.where(np.isnan(thick[0,:,:,:]),0,thick[0,:,:,:])/100.
    aweight[b,:,:,:] = mask[b,:,:,:]*area[np.newaxis,:,:] 

In [6]:
#Calculate basin-average annual time series
tbas = np.zeros((nyears,len(ut.basin)))
ttime = np.arange(nyears)
months = np.arange(0,12)

c = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname)
    time = ds['time_centered'].values
    temp = ds['thetao'].values
    ds.close()
    
    year0 = int(fname[-27:-23])
    
    tb = np.zeros((len(ut.basin)))
    ny = int(len(time)/12)
    for y in np.arange(0,ny):
        for b,bas in enumerate(ut.basin):
            for m,mm in enumerate(months):
                tbb = np.nansum(temp[m+12*y,:,:,:]*aweight[b,:,:,:])/np.nansum(aweight[b,:,:,:])
                tb[b] += tbb*secs[m]
            tbas[c,b] = tb[b]/sum(secs)
        print(year0+y,c,tbas[c,:])
        tb = np.zeros((len(ut.basin)))
        c += 1

1850 0 [ 0.26118002  1.43502922  2.45091204 -1.5162289   0.10452265]
1851 1 [ 0.23456867  1.74010643  2.51300198 -1.49069721  0.18712991]
1852 2 [ 0.243741    1.49206737  2.53659533 -1.51732994  0.14620583]
1853 3 [ 0.26907567  1.40006475  2.55081797 -1.57316726  0.07210651]
1854 4 [ 0.21952625  1.44276231  2.55035899 -1.58823267  0.03790708]
1855 5 [ 0.15584439  1.7390174   2.54025081 -1.52912061 -0.08806093]
1856 6 [ 0.24963334  1.87509838  2.49717286 -1.46627775 -0.09997161]
1857 7 [ 0.30695886  1.49294861  2.51564772 -1.46164821 -0.09028759]
1858 8 [ 0.36665841  1.52864425  2.47472132 -1.44650827 -0.21711695]
1859 9 [ 0.48062513  1.51728653  2.42900112 -1.35935455 -0.30089888]
1860 10 [ 0.54372166  1.5220688   2.35828357 -1.21770141 -0.30650033]
1861 11 [ 0.61184185  1.44845194  2.33677951 -1.11030423 -0.20014648]
1862 12 [ 0.58935329  1.0792017   2.28976269 -1.13528723 -0.16087827]
1863 13 [ 0.55819581  0.75444064  2.27538386 -1.1712463  -0.21695234]
1864 14 [ 0.49889457  0.860581

In [7]:
temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime,'basin':ut.basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})

ds = xr.Dataset({'temp':temp2})
ds.to_netcdf(f'../data/temperature_{run}.nc',mode='w')
ds.close()

In [8]:
mask2 = xr.DataArray(mask,dims=('basin','olevel','y','x'),coords={'basin':ut.basin,'olevel':levmid},attrs={'long_name':'basin mask'})
lat2  = xr.DataArray(lat,dims=('y','x'))
lon2  = xr.DataArray(lon,dims=('y','x'))

ds = xr.Dataset({'mask':mask2,'lat':lat2,'lon':lon2})
ds.to_netcdf(f'../data/basinmask.nc',mode='w')
ds.close()